In [ ]:
import pandas as pd
from typing import List
from pathlib import Path
from itertools import chain
import matplotlib.pyplot as plt
from sklearn.preprocessing import *


In [ ]:
source = Path("runs/predict")
paths = [folder for folder in source.rglob('*') if folder.is_dir()]
csv = list(chain.from_iterable(path.glob(f"{path.name}_merged_output.csv") for path in paths))


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from typing import List

# Function to visualize data from multiple CSV files / 여러 CSV 파일의 데이터를 시각화하는 함수


def outlier_visualize_combined(csv_paths: List[Path]) -> None:
    num_files = len(csv_paths)

    # Create a figure for subplots / 서브플롯을 위한 전체 figure 생성
    fig, axes = plt.subplots(3, num_files, figsize=(5 * num_files, 15))

    # Iterate through each CSV file / 각 CSV 파일을 순회
    for idx, path in enumerate(csv_paths):
        data = pd.read_csv(path)

        # First row: Width histogram / 첫 번째 행: Width 히스토그램
        axes[0, idx].hist(data['Width'], bins=50, alpha=0.6, color='blue')
        axes[0, idx].set_title(f'Width Histogram - {path.name}')
        axes[0, idx].set_xlabel('Width')
        axes[0, idx].set_ylabel('Frequency')

        # Second row: Height histogram / 두 번째 행: Height 히스토그램
        axes[1, idx].hist(data['Height'], bins=50, alpha=0.6, color='green')
        axes[1, idx].set_title(f'Height Histogram - {path.name}')
        axes[1, idx].set_xlabel('Height')
        axes[1, idx].set_ylabel('Frequency')

        # Third row: Scatter plot / 세 번째 행: Width vs Height 스캐터 플롯
        axes[2, idx].scatter(data['Width'], data['Height'], alpha=0.3, c='red')
        axes[2, idx].set_title(f'Scatter Plot - {path.name}')
        axes[2, idx].set_xlabel('Width')
        axes[2, idx].set_ylabel('Height')
        axes[2, idx].grid(True)

    # Adjust layout and display the plots / 레이아웃 조정 및 플롯 표시
    plt.tight_layout()
    plt.show()


In [ ]:
outlier_visualize_combined(csv)


In [ ]:
# Outlier removal function / 이상치 제거 함수
def remove_outliers(df: pd.DataFrame, column: str) -> pd.DataFrame:
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Function to visualize boxplots for multiple CSV files / 여러 CSV 파일의 박스플롯을 시각화하는 함수


def boxplot_visualize_combined(csv_paths: List[Path]) -> None:
    num_files = len(csv_paths)

    # Create a figure for subplots with 2 rows / 서브플롯을 위한 2행 전체 figure 생성
    fig, axes = plt.subplots(2, num_files, figsize=(5 * num_files, 10))

    # Iterate through each CSV file / 각 CSV 파일을 순회
    for idx, path in enumerate(csv_paths):
        data = pd.read_csv(path)

        # First row: Boxplot for Width and Height before outlier removal / 첫 번째 행: 이상치 제거 전 Width와 Height에 대한 박스플롯
        data[['Width', 'Height']].boxplot(ax=axes[0, idx])
        axes[0, idx].set_title(f'Boxplot Before Outlier Removal - {path.name}')

        # Remove outliers for Width and Height / Width 및 Height에 대한 이상치 제거
        data_no_outliers = data.copy()
        data_no_outliers = remove_outliers(data_no_outliers, 'Width')
        data_no_outliers = remove_outliers(data_no_outliers, 'Height')

        # Second row: Boxplot for Width and Height after outlier removal / 두 번째 행: 이상치 제거 후 Width와 Height에 대한 박스플롯
        data_no_outliers[['Width', 'Height']].boxplot(ax=axes[1, idx])
        axes[1, idx].set_title(f'Boxplot After Outlier Removal - {path.name}')

    # Adjust layout and display the plots / 레이아웃 조정 및 플롯 표시
    plt.tight_layout()
    plt.show()


In [ ]:
boxplot_visualize_combined(csv)


In [ ]:
source = Path("runs/predict2")
paths = [folder for folder in source.rglob('*') if folder.is_dir()]
csv = list(chain.from_iterable(path.glob(f"{path.name}_merged_output.csv") for path in paths))


In [ ]:
outlier_visualize_combined(csv)


In [ ]:
boxplot_visualize_combined(csv)


In [ ]:
from pathlib import Path
import pandas as pd

# 병합할 CSV 파일들이 있는 디렉토리 경로를 입력하세요.
input_folder = Path('./runs/predict/exp')

# 병합된 CSV 파일을 저장할 파일명을 입력하세요.
output_file = '_merged_output.csv'

# 지정된 디렉토리 내의 모든 CSV 파일 목록을 가져옵니다.
csv_files = sorted(input_folder.glob('*.csv'))

# 데이터프레임을 저장할 리스트를 초기화합니다.
dataframes = []

# CSV 파일들을 순회하며 데이터프레임으로 읽어들입니다.
for idx, file in enumerate(csv_files):
    if idx == 0:
        # 첫 번째 파일은 헤더를 포함하여 읽어옵니다.
        df = pd.read_csv(file)
    else:
        # 이후 파일들은 헤더를 스킵하고 읽어옵니다.
        df = pd.read_csv(file, skiprows=1, header=None)
    dataframes.append(df)

# 데이터프레임들을 하나로 병합합니다.
merged_df = pd.concat(dataframes, ignore_index=True)

# 첫 번째 데이터프레임의 컬럼명을 사용하도록 헤더를 설정합니다.
merged_df.columns = dataframes[0].columns

# 병합된 데이터프레임을 CSV 파일로 저장합니다.
output_file = input_folder.name / output_file

merged_df.to_csv(output_file, index=False, header=True)


In [ ]:
source = Path("runs/predict2/predict2_merged_output.csv")


In [ ]:
data = pd.read_csv(source)
data[['Width', 'Height']].boxplot()


In [ ]:
data_no_outliers = data.copy()
data_no_outliers = remove_outliers(data_no_outliers, 'Width')
data_no_outliers = remove_outliers(data_no_outliers, 'Height')


In [ ]:
data_no_outliers[['Width', 'Height']].boxplot()


In [ ]:
# from ultralytics import YOLO

# model = YOLO("./fish_size/checkpoints/best.pt")

# metrics = model.val()


In [ ]:
from ultralytics import YOLO
from pathlib import Path

model = YOLO('./fish_size/checkpoints/best.pt')

source = Path('./dataset/test_img')
paths = [folder for folder in source.rglob('*') if folder.is_dir()]

for path in paths:
    results = model.predict(
        source=path,
        save=True,
        save_txt=True,
        stream=True,
        verbose=False,
        conf=0.75,
        iou=0.75
    )

    for result in results:
        print(result)
